# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [ ]:
!pip install "tqdm==4.43.0"

     |████████████████████████████████| 59 kB 2.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
!pip install datasets

     |████████████████████████████████| 542 kB 5.3 MB/s 
     |████████████████████████████████| 118 kB 55.9 MB/s 
     |████████████████████████████████| 243 kB 53.4 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 636 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.14
    Uninstalling huggingface-hub-0.0.14:
      Successfully uninstalled huggingface-hub-0.0.14


In [ ]:
!pip install rouge_score

In [ ]:
# Importing necessary librairies
import datasets
from datasets import load_metric
import io
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import pandas as pd
import torch 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm, trange
import seaborn as sns
from sklearn.model_selection import train_test_split

## 1.2. Importing personal drive to file in order to later load the data and the saved fine tuned model


In [ ]:
# Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Data formatting

## 2.1. Loading clean data 

In [ ]:
# We load the cleaned data from a pickle format
file ='/content/drive/MyDrive/Colab Notebooks/Data/final_clean.pickle'
df = pd.read_pickle(file)

In [ ]:
# Looking at the data
df

,description_narrative,TLDR,text_clean,text_embedding,summary_clean,summary_embedding,labels,labels_idx_list,doc_label
0,The training improved women’s knowledge on the...,The training improved women’s knowledge on the...,[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...",[The training improved women’s knowledge on th...,"[[0.76747984, -0.1894493, 0.51285785, -0.02116...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[0],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Illegal oil refining in the Niger Delta is inc...,CEHRD in an effort to create awareness on the ...,[Illegal oil refining in the Niger Delta is in...,"[[-0.26888534, -0.6092957, -0.23817927, -1.167...",[CEHRD in an effort to create awareness on the...,"[[-0.43648192, -0.071132354, -0.18081759, -0.4...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,CEHRD successfully set-up 4 formal and 4 infor...,CEHRD set-up the environmental clubs with the ...,[CEHRD successfully set-up 4 formal and 4 info...,"[[-0.48964322, -1.2085572, 1.0374501, 0.021369...",[CEHRD set-up the environmental clubs with the...,"[[-0.15335692, -0.29430717, 0.58692193, -1.126...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[10],"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,PROJECT NAME : Delivering Accelerated Family P...,Delivering Accelerated Family Planning in Paki...,"[PROJECT NAME :, Delivering Accelerated Family...","[[-0.23698506, 0.15983863, -0.071198896, -1.20...",[Delivering Accelerated Family Planning in Pak...,"[[-0.50266284, -1.2923895, 0.4206834, -1.25673...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[1],"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
4,Road traffic injuries are world's eighth leadi...,The Cardiff Trauma Pack Research and Develop...,[Road traffic injuries are world's eighth lead...,"[[-0.4666822, -1.19177, 0.9945381, -0.95759714...",[ The Cardiff Trauma Pack Research and Develo...,"[[-0.4679708, -0.43502182, 0.8388376, -1.47547...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[4, 5]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ..."
...,...,...,...,...,...,...,...,...,...
2980,Metta has been responding to the humanitarian ...,"ProjectGoal: To improve the condition of 2,854...",[Metta has been responding to the humanitarian...,"[[-0.6875755, -1.0828758, 0.32236806, -1.33987...","[ProjectGoal:, To improve the condition of 2,8...","[[-0.58209735, 0.027911462, 0.29285246, -1.098...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[7, 8]","[2980, 2980, 2980, 2980, 2980, 2980, 2980, 298..."
2981,"Destined Women is local not for profit, non-re...",GOAL: Contribute towards changing the socioeco...,"[Destined Women is local not for profit, non-r...","[[-0.3787009, -0.7928037, 0.16012278, -0.16681...","[GOAL:, Contribute towards changing the socioe...","[[-0.2746246, 0.04018628, 0.4832729, -0.996766...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1, 7]","[2981, 2981, 2981, 2981, 2981, 2981, 2981, 298..."
2982,The project will empower secondary school stud...,The project will empower secondary school stud...,[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...",[The project will empower secondary school stu...,"[[0.3152278, -0.62204534, 1.0845532, -0.392865...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2982, 2982, 2982, 2982, 2982, 2982, 2982, 2982]"
2983,The project has been specifically designed to ...,The project has been specifically designed to ...,[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...",[The project has been specifically designed to...,"[[0.2448853, -0.043037124, 0.5578783, -0.51006...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0],"[2983, 2983, 2983, 2983, 2983, 2983]"


## 2.2. Creating the train and test sets in the right format

### 2.2.1. Creating the train set, with each row being a sentence 

In [ ]:
# Splitting the dataframe into a train and a test set, keeping 20% for the test set. 
train_dataset, test_dataset = train_test_split(df, test_size=0.2)

# We wish to split each sentence individually for the purpose of classification. 
#for this reason, we create three empty lists to later feed them with the individual sentences, associated with their label and document label 
sentences, labels, document_labels = [], [], [] 

# We do the below to feed the empty lists
#looping through the embeddings, the document labels, the text and the target labels
for doc_x, doc_y, doc_label in zip(train_dataset['text_clean'], 
                                   train_dataset['labels'], 
                                   train_dataset['doc_label']):
                                          
#looping through each item within the text, the target labels, the document labels
    for i, (sent, target, label) in enumerate(zip(doc_x, doc_y, doc_label)):
        
        #appending each item to the empty lists previously created 
        sentences.append(sent) #this will be used to train the data points 
        labels.append(target) #this will be used to train the data points
        document_labels.append(label) #this will be used to retrieve the intial document the sentence belongs to 

# We then create a dataframe out of the created lists 
train_dataset = pd.DataFrame({'sentence': sentences,
                             'label' :  labels,
                             'document_label': document_labels,
                             })

# For precision, we convert the current float labels to integers 
#creation of an empty list
new_label = []

#looping in the label column, converting each label to an integer and appending to the new_label list
for i in train_dataset['label']:
  i = int(i)
  new_label.append(i)  

#updating the label column with the new_label list
train_dataset['label'] = new_label

# Viewing the pre-formatted train_dataset
train_dataset

,sentence,label,document_label
0,"The objective of the Investing in Maternal, Ch...",1,1206
1,This project consists of four components.,0,1206
2,Component one will support key interventions t...,0,1206
3,There are three subcomponents: (1) Supporting ...,0,1206
4,The second component will promote adolescent h...,0,1206
...,...,...,...
24929,This component will contribute to the Governme...,0,440
24930,The second component of the project is improvi...,0,440
24931,This component will be implemented by Pernambu...,0,440
24932,The third component of the project is water su...,0,440


### 2.2.2. Creating the test set, with each row being a sentence



In [ ]:
# We repeat the same process for the test_dataset

# Creating empty lists 
sentences, labels, document_labels = [], [], [] 

#looping through the embeddings, the document labels, the text and the target labels
for doc_x, doc_y, doc_label in zip(test_dataset['text_clean'], 
                                   test_dataset['labels'], 
                                   test_dataset['doc_label']):
                                          
    #looping through each item within the text, the target labels, the document labels
    for i, (sent, target, label) in enumerate(zip(doc_x, doc_y, doc_label )):
        
        #appending each item to the empty lists previously created 
        sentences.append(sent) #this will be used to train the data points 
        labels.append(target) #this will be used to train the data points
        document_labels.append(label) #this will be used to retrieve the intial document the sentence belongs to 

# Creating a new test_dataset dataframe with appropriate lists 
test_dataset = pd.DataFrame({'sentence': sentences,
                             'label' :  labels,
                             'document_label': document_labels,
                             })

# Converting float labels to integers 
new_label = []

for i in test_dataset['label']:
  i = int(i)
  new_label.append(i)   

#updating the label column with the new_label list
test_dataset['label'] = new_label

# Viewing the pre-formatted train_dataset
test_dataset

,sentence,label,document_label
0,The objectives of the Water Services and Insti...,1,1192
1,The Project has four components.,0,1192
2,"Sector policy, regulations and institutions co...",0,1192
3,Support for water supply and sewerage sector r...,0,1192
4,Project management and preparation of future i...,0,1192
...,...,...,...
6230,There are 5 Foshan components and 2 Jiangmen c...,0,421
6231,The Foshan components are: (a) Wastewater Mana...,0,421
6232,Expansion of Zhen'an wastewater treatment plan...,0,421
6233,Construction of a 400 tons/day capacity centra...,0,421


# 3. Loading the fine-tuned BertForSequenceClassification model



We'll be using [BertForSequenceClassification](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#bertforsequenceclassification). This is the normal BERT model with an added single linear layer on top for classification that we will use as a sentence classifier. As we feed input data, the entire pre-trained BERT model and the additional untrained classification layer is trained on our specific task. 


In [ ]:
# Loading the previsouly trained and fine-tuned model using the from_pretrained method of hugginface's transformer library
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/BERT_fine_tuned_uncased_2_epochs")

# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## 4. Performance On Test Set

### 4.1 Prepare the dataset

BERT requires a specific format for its input, namely: 

- Tokenize the sentences at a word level
- Special tokens [CLS] and [SEP] respectively at the begggining and end of each sentence (i.e. input) 
- Map all tokens to their input IDs 
- Pad all sentences for each input to match the same length; the one of the longest sentence. 
- Add Attention mask to differentitate normal tokens from padded tokens.

We implement these pre-processing steps in the below cells. 

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) #we choose bert-base-uncased as it has shown to be more effective for classification purposes than the bert-base-cased version.

# Now computing the longest sentence's length to later pad to that length 
#setting the max length to 0
max_len = 0

#create sentence and label lists from test dataset
sentences = test_dataset.sentence.values
labels = test_dataset.label.values

#for every sentence...
for sent in sentences:

    #tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    #update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

#printing the maximum sentence length. 
print('Max sentence length: ', max_len)

Loading BERT tokenizer...



Max sentence length:  199


Now that we have the maximum sentence length, we can pre-process all the sentences as per defined above. 

We use the encode_plus method of huggingface's BERT tokenizer to carry these steps.


In [ ]:
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(test_dataset.shape[0]))

# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Number of test sentences: 6,235



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## 4.2. Evaluate on Test Set


## 4.2.1. Making predictions on the test set


With the test set prepared, we can apply our fine-tuned model to generate predictions on the test set.

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions  = []

# Predict 
for batch in prediction_dataloader:
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions.
    result = model(b_input_ids, 
                   token_type_ids=None, 
                   attention_mask=b_input_mask,
                   return_dict=True)
    
  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  
  # Store predictions and true labels
  predictions.append(logits)

print('    DONE.')

Predicting labels for 6,235 test sentences...
    DONE.


## 4.2.2. Computing the ROUGE metrics to assess the model's performance

### 4.2.2.1. Formatting the data to allow metric computation

In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()


# Adding flat_prediction to the test_datasets
test_dataset['prediction'] = flat_predictions

# Create a new dataframe which groups the predictions based on their document_label to map the prediction to the original text 
df_summaries = test_dataset.groupby('document_label').agg(list)
#resetting the index
df_summaries = df_summaries.reset_index()

# Now we wish to extract the summaries only 
# Keeping the sentences that were predicted with 1
#creating empty lists
predicted_labels, predicted_sentences, actual_labels, actual_sentences, doc_labels =[],[],[],[],[]

#looping in every line of the df_summaries simultaneously 
for predicted_label, predicted_text, actual_label, doc_label in zip(df_summaries['prediction'], 
                                                                    df_summaries['sentence'], 
                                                                    df_summaries['label'], 
                                                                    df_summaries['document_label']):
    #looping in each item of each line of df_summaries
    for pred_label, pred_sent, act_label in zip(predicted_label, 
                                                              predicted_text, 
                                                              actual_label):
        #setting the condition that only if the item equals 1 in the predicted_labels
        #assing the text, the predicted label, the actual label and the document label to the lists 
        if pred_label == 1:
            predicted_labels.append(pred_label)
            predicted_sentences.append(pred_sent)
            actual_labels.append(act_label)
            doc_labels.append(doc_label)

# Creating a new dataframe with only the predicted sentences kept, against the actual label 
df_summaries = pd.DataFrame({'predicted_label': predicted_labels,
                             'predicted_summary': predicted_sentences,
                             'reference_label' : actual_labels,
                             'document_label': doc_labels})

#grouping by the document label and resetting the index 
df_summaries = df_summaries.groupby('document_label').agg(list).reset_index()

# Adding the actual summaries for each document to the dataframe
# This is critical for the evaluation 

#creataing an empty list 
actual_summaries = []

#looping though the document labels in the df_summaries 
for label in df_summaries['document_label']:
  #looping through both the indexes and the text of the original summaries in the initial dataframe
  for index, summ in zip(df.index, df['summary_clean']):
        #setting the condition that if the index of the actual summary in the original document equals 
        #to the label of the document in the results (i.e. summaries) dataframe 
        #append the summary to the empty list
        if label == index:
            actual_summaries.append(summ)

# Adding the list to the df_summaries dataframe
df_summaries['reference_summary'] = actual_summaries



### 4.2.2.1. Computing ROUGE scores

In [ ]:
# Computing rouge scores for the results 

#load Huggingface's rouge metric 
metric = load_metric("rouge")

#creating empty lists for the predicted and the actual summaries 
predicted_summ = []
label_summ = []

#looping through both the predicted and the actual summaries 
for doc_pred, doc_label in zip(df_summaries['predicted_summary'], df_summaries['reference_summary']):
    #looping through each sentence within the predicted and the actual summaries  
    for pred_summ, labelled_summ in zip(doc_pred, doc_label):
        #appending these to the empty lists 
        predicted_summ.append(pred_summ)
        label_summ.append(labelled_summ)

#computing and saving the rouge scores in rouge_scores object 
rouge_scores = metric.compute(predictions=predicted_summ, references=label_summ)

#printing the scores 
print(rouge_scores)

{'rouge1': AggregateScore(low=Score(precision=0.818276159029746, recall=0.8750333292386197, fmeasure=0.833080300101144), mid=Score(precision=0.8426020570817803, recall=0.8985443150590715, fmeasure=0.8574155809678228), high=Score(precision=0.866314148140687, recall=0.9189755476847123, fmeasure=0.8798736184369884)), 'rouge2': AggregateScore(low=Score(precision=0.7945191433789944, recall=0.8442218384736072, fmeasure=0.8093057309292104), mid=Score(precision=0.821124054710513, recall=0.8714184967713516, fmeasure=0.8358565713630062), high=Score(precision=0.8498249538102607, recall=0.8982324659522973, fmeasure=0.8632517414296937)), 'rougeL': AggregateScore(low=Score(precision=0.8103604490861679, recall=0.8674556248999888, fmeasure=0.8252567688649324), mid=Score(precision=0.835775473216379, recall=0.8903833435823651, fmeasure=0.8505487134241715), high=Score(precision=0.8612813759008843, recall=0.9124262200627823, fmeasure=0.8758159468309867)), 'rougeLsum': AggregateScore(low=Score(precision=0.